In [ ]:
import sys
sys.path.append('../')
import hicap_analysis.wells as wo
from hicap_analysis.geoprocessing import Geoprocess

import os
from pathlib import Path
import pandas as pd
import geopandas as gpd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

# Set some paths
shapefiles and a testing csv file are in the tests directory.

In [ ]:
homepath = Path(os.getcwd())
datapath = homepath.parents[0] / 'hicap_analysis' / 'tests' / 'data'

# Make a geoprocessor object

The geoprocessor object needs a shapefile with the desired catchments
and one with the corresponding streamlines.  catch_idx and stream_idx
are optional and will be used to set the indices of the returned
geodataframes if provided.  

In [ ]:
geopro = Geoprocess(datapath / 'WWAP_110507.shp', 
                    datapath / 'WWAP_ValleySegments_080907.shp', 
                    catch_idx='ADJ_SEGMNT', 
                    stream_idx='ADJ_SEGMNT')

# make a well dataframe

The geoprocess will take a list of [lat, long] pairs, a geodataframe, 
or a dataframe with the lat, long of target wells.  In this test, I'm 
passing a dataframe and the processor makes a geodataframe and reprojects
to the same projection as the Geoprocess object.

In [ ]:
well = [{'name': 'testwell0',
    'lat': 44.979953,
    'long': -84.625023,
    'rate': 70,
    'depth': 80},
   {'name': 'testwell1',
    'lat': 44.99,
    'long': -84.64,
    'rate': 70,
    'depth': 80}]  #gpm and ft

well_df=pd.DataFrame(well)

## pass the dataframe to the geoprocessor

The get_geometries() returns a WellGeometry objects.

In [ ]:
well_list = geopro.get_geometries(well_df)
well_list

## WellGeometry objects

The attributes are:

- name
- geodataframe of catchment with the well (home_df)
- geodataframe of the neighboring catchments (neighbors_df)
- geodataframe of the streamlines for the catchments (streams_df)
- dataframe of the closest point on the stream for each catchment
  and apportionment for inverse-distance and inverse-distance-squared

In [ ]:
# testwell0 should match the table from SIR
home = well_list[0].home_df.copy()
nearest = well_list[0].close_points_df.copy()

In [ ]:
home

In [ ]:
nearest

# Compare to the table of results (Table 2) in SIR 2009-5003
This is the test that is coded in tests/test_calcs.py

In [ ]:
# need to call the hunt99 function
time = 5. * 365.25  # 5 years
# pumping is 70 gpm; 1 gpm = 0.0022280093 cfs
Q = well_df.loc[0,'rate'] * 0.0022280093 * 3600 * 24  # rate in CFD for function.
T = home.loc[11967, 'MEDIAN_T']
S = 0.01
streambed = home.loc[11967, 'EST_Kv_W']/well_df.loc[0, 'depth']

In [ ]:
# hunt99 returns CFS need to convert to GPM for table
nearest['analytical_removal'] = nearest['distance'].apply(lambda dist: wo._hunt99(T, S, time, dist, Q, streambed)* 448.83116885)
nearest['valley_seg_removal'] = nearest['apportionment'] * nearest['analytical_removal']
nearest['percent'] = nearest['apportionment'] * 100.

In [ ]:
check_df = pd.read_csv(datapath / 'SIR2009_5003_Table2_Batch.csv', dtype=float)
check_df.set_index('Valley_segment', inplace=True)

In [ ]:
tol = 0.01
np.testing.assert_allclose(nearest['percent'].values, check_df['Removal_percent'].values, atol=tol)
tol = 0.04
np.testing.assert_allclose(nearest['analytical_removal'].values, check_df['Analytical_removal_gpm'].values, atol=tol)
tol = 0.01
np.testing.assert_allclose(nearest['valley_seg_removal'].values, check_df['Estimated_removal_gpm'].values, atol=tol)

# Note you can easily plot the geodataframes or write as shapefile

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
well_list[0].neighbors_df.plot(ax=ax, color='lightgray', edgecolor='darkgray')
home.plot(ax=ax, color='yellow', edgecolor='darkgray')
well_list[0].well_df.plot(ax=ax, color='red')
well_list[0].streams_df.plot(ax=ax, color='blue')
nearest.plot(ax=ax, color='orange')
ax.tick_params(direction='in')

In [ ]:
nearest.to_file(homepath / 'output' / 'nearest.shp')

In [ ]:
home